## Project Name: Predicting US personal medical insurance charges using machine learning.

### OUTCOMES

- 1. load the dataset

- 2. Y data Profiling (Automatic EDA)
    - Basic Understanding the dataset
    - drop the column
    - Rename or Simply columns
    - Correlation Numberical features VS Target Variable

- 3. Train Test Split
    - Seperate Features Matrix (X) and Target Variable (y)
    - Train Test Split

- 4. Create Pipeline For Our model

    - 1. Pipeline for Numerical Features
    - 2. Pipeline for Categorical Features
    - 3. Combine them (numerical and catagorical Features)
    - 4. Define Model
        - Base learner 
        - Fit the base learner with Our model

- 5. Model Training and Evaluation
    - Prediction and visualization


- 6. Cross Validation

- 7. Grid Search CV on Our model
    - To Get the best Parameter

- 8. Randomized Search CV
    - To Get the best Parameter

- 9. Comapre Which Parameters is Best for Our model.
    - 9.1: Prediction and Evaluation for Grid search:
    - 9.2: Prediction and Evaluation for Random Search cv:

- 10. Save Model with Best Parameters

- 11. MLFlow Introduce
    - 11.1: Discover MLFlow

    - 11.2: Mlflow with RandomForest model

    - 11.3: Mlflow with our best model

In [57]:
### === import necessary libraries === ###

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# EDA
from ydata_profiling import ProfileReport


## Sklearn Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


# Model 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor, StackingRegressor

#### Evaluation Metrices
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

### 1. load the dataset

In [172]:
# load the dataset
df = pd.read_csv("insurance.csv")

df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
df.columns

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')

In [174]:

print("Unique values per column:")
for col in df.columns:
    print(col, ":", df[col].nunique())

Unique values per column:
age : 47
sex : 2
bmi : 548
children : 6
smoker : 2
region : 4
charges : 1337


In [184]:
df['age'].min()

np.int64(18)

In [180]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [183]:
df['bmi'].max()

np.float64(53.13)

In [186]:
df['children'].max()

np.int64(5)

In [187]:
df['region']

0       southwest
1       southeast
2       southeast
3       northwest
4       northwest
          ...    
1333    northwest
1334    northeast
1335    southeast
1336    southwest
1337    northwest
Name: region, Length: 1338, dtype: object

In [59]:
# Shape
df.shape

(1338, 7)

### 2. Y data Profiling (Automatic EDA)

In [6]:
# y-data profiling

profile = ProfileReport(df, title="Health Insurance Cost Analysis", explorative = True)

# convert csv file to html 
profile.to_file("ydata_insurance.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 143.18it/s]


In [60]:
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


#### 2.1: Basic Understanding the dataset

- 1. Dataset statistics:
    - Total Features : 7
    - Total Sample   : 1338

    - Numerical Features:
        - [age, bmi, children, charges]

    - Categorical Features:
        - [sex, smoker, region]

- 2. Columns & Their Meaning:

    - 1. age:
        - Age of primary beneficiary.
    
    - 2. sex:
        - Gender either male or female.
    
    - 3. bmi:
        - a quick measure of body fat based on height and weight.

        - Body Mass Index (weight(kg)/height²(m^2)).

    - 4. children:
        - Number of children or dependents covered by health insurance.

    - 5. smoker:
        - smoker Status either Yes or NO.

    - 6. region:
        - The beneficiary's residentia area in US.

    - 7. charges:
        - Individual medical costs billed by health insurance.


- ydata Profiling (Automatical EDA) we get some decision:
- ydata Profiling (Automatical EDA) we get some decision:

- 1. Numerical Features
    - [age, bmi, children]

- 2. Categorical Features
    - [sex, smoker, region]

- 3. Target Features
    - [ charges ]


- NOTE: 

    - "age" is highly overall correlated with "charges".
    
    - "smoker" is highly overall correlated with "charges".

    - There is one Duplicate row
        - So, We Keeps the first occurrence.


#### 2.2:  Keeps the first occurrence 

In [61]:
df.shape

(1338, 7)

In [62]:
# keeps the first occurance
df = df.drop_duplicates()

df.shape

(1337, 7)

In [63]:
df.head(2)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


#### 2.3: Rename or Simply columns

In [64]:
# Rename So we easily understand our features or columns

df.columns = [
   'person_age', 'person_gender', 'person_bmi', 'num_children', 'is_smoker',
   'region', 'medical_cost'
]

df.head(2)

,person_age,person_gender,person_bmi,num_children,is_smoker,region,medical_cost
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [65]:
#    - Show number of unique values per column.

print("Unique values per column:")
for col in df.columns:
    print(col, ":", df[col].nunique())

Unique values per column:
person_age : 47
person_gender : 2
person_bmi : 548
num_children : 6
is_smoker : 2
region : 4
medical_cost : 1337


In [66]:
#    - Show missing value summary for each column.

df.isnull().sum()

person_age       0
person_gender    0
person_bmi       0
num_children     0
is_smoker        0
region           0
medical_cost     0
dtype: int64

In [67]:
# Summary statistics for numerical coulumns

df.describe().T

,count,mean,std,min,25%,50%,75%,max
person_age,1337.0,39.222139,14.044333,18.0000,27.000,39.0000,51.00000,64.00000
person_bmi,1337.0,30.663452,6.100468,15.9600,26.290,30.4000,34.70000,53.13000
num_children,1337.0,1.095737,1.205571,0.0000,0.000,1.0000,2.00000,5.00000
medical_cost,1337.0,13279.121487,12110.359656,1121.8739,4746.344,9386.1613,16657.71745,63770.42801


#### 2.4: Correlation Numberical features VS Target Variable

In [68]:
# Numerical vs Target Correlation

corr_target = df.select_dtypes(include= np.number).corr()['medical_cost'].sort_values(ascending=False)

print("Correlation Between Numerical Features and Target Variable: \n", corr_target)

Correlation Between Numerical Features and Target Variable: 
 medical_cost    1.000000
person_age      0.298308
person_bmi      0.198401
num_children    0.067389
Name: medical_cost, dtype: float64


###  3. Train Test Split
- 3.1: Seperate Features Matrix (X) and Target Variable (y)
- 3.2: Train Test Split

#### 3.1: Seperate Features Matrix (X) and Target Variable (y)

In [69]:
# Features matrix
X = df.drop('medical_cost', axis = 1)

# Target Variable
y = df['medical_cost']

X.shape, y.shape

((1337, 6), (1337,))

In [73]:
# Numerical Features
numeric_features = X.select_dtypes(include= ['int64', 'float64']).columns

# Categorical Features

categorical_features = X.select_dtypes(include=['object']).columns

numeric_features

Index(['person_age', 'person_bmi', 'num_children'], dtype='object')

In [74]:
categorical_features

Index(['person_gender', 'is_smoker', 'region'], dtype='object')

In [24]:
#  Univariate Analysis on numerical column

import matplotlib.pyplot as plt

# # Histograms 
df[numeric_features].hist(bins=20, figsize=(10, 6))
plt.tight_layout()
plt.show()

In [ ]:
# Write Countplots code for key categorical variables

for c in categorical_features:

    plt.figure(figsize = (5,4))

    # df[c].value_counts().plot(kind = 'bar')

    sns.countplot(data=df, x = df[c])

    plt.title(f"Value counts for {c}")

    plt.ylabel("count")

    plt.xticks(rotation = 0) # [Default - 90]

    plt.tight_layout()

    plt.show()


In [28]:
# Counts
print("Target Distribution:")
print(df['medical_cost'].describe())

# plot 
plt.figure(figsize=(8,5))
sns.histplot(df['medical_cost'], kde=True, bins=50, color='skyblue')
plt.title('Distribution of Insurance Charges')
plt.xlabel('Charges ($)')
plt.ylabel('Frequency')
plt.show()

Target Distribution:
count     1337.000000
mean     13279.121487
std      12110.359656
min       1121.873900
25%       4746.344000
50%       9386.161300
75%      16657.717450
max      63770.428010
Name: medical_cost, dtype: float64


In [30]:
df.head(2)

,person_age,person_gender,person_bmi,num_children,is_smoker,region,medical_cost
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523


In [49]:
# # Bivariate Analysis Numerical columns with Target 

# import matplotlib.pyplot as plt
# import seaborn as sns

# numeric_cols = ['person_age', 'person_bmi', 'num_children']
# target_col = 'medical_cost'

# corr_matrix = df[numeric_cols + [target_col]].corr()

# plt.figure(figsize=(8,6))
# sns.heatmap(
#     corr_matrix,
#     annot=True,
#     cmap='coolwarm',
#     fmt='.2f',
#     linewidths=0.5
# )

# plt.title("Correlation Heatmap for Numerical Features and Target")
# plt.xticks(rotation=90)
# plt.tight_layout()
# plt.show()


#### 3.2: Train Test Split

In [35]:
df.shape

(1337, 7)

In [75]:
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

# this is a Regression problem that way we can't use "stratify"

((1069, 6), (268, 6))

### 4. Create Pipeline For Our model

- 4.1: Pipeline for Numerical Features

- 4.2: Pipeline for Categorical Features

- 4.3: Combine them (numerical and catagorical Features)

- 4.4: Define Model
        - Base learner 
        - Fit the base learner with Our model

#### 4.1: Pipeline for Numerical Features
- imputation
- scaler

In [76]:
# Pipeline for numerical features

num_transformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='median')),
        ('scalar', StandardScaler())
    ]
)

num_transformer

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('scalar', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'median'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a f

#### 4.2: Pipeline for Categorical Features
- imputation
- OHE

In [77]:
# Pipeline for Categorical features

cat_transformer = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]
)

cat_transformer

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('imputer', ...), ('encoder', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'most_frequent'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",None
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation

#### 4.3. Combine them (numerical and catagorical Features)

In [78]:
# combine them
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer,numeric_features),
        
        ('cat', cat_transformer, categorical_features)
    ]
)
preprocessor

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `

#### 4. Define Model

- 4.1: We use base learner 
    - LinearRegression()
    - RandomForestRegressor()
    - GradientBoostingRegressor()

- 4.2: Fit the base learner with Our model 
    - VotingRegressor()
    - StackingRegressor()

##### 4.1: We use base learner
- LinearRegression()
- RandomForestRegressor()
- GradientBoostingRegressor()

In [84]:
from sklearn.svm import SVR

In [97]:
# Base Learner

reg_lr = LinearRegression()

reg_rf = RandomForestRegressor(n_estimators=100, random_state=42)

reg_gb = GradientBoostingRegressor(n_estimators=100, random_state=42)

reg_svr = SVR(kernel = "rbf", C = 300, epsilon = 0.1, gamma = "scale")


##### 4.2: Fit the base learner with Our model 
- VotingRegressor()
- StackingRegressor()

In [98]:
# voting regressor

voting_reg = VotingRegressor(
    estimators=[
        ('lr', reg_lr),
        ('svr',reg_svr),
        ('rf', reg_rf),
        ('gb', reg_gb)
    ]
)
voting_reg

,"estimators estimators: list of (str, estimator) tuplesInvoking the ``fit`` method on the ``VotingRegressor`` will fit clonesof those original estimators that will be stored in the class attribute``self.estimators_``. An estimator can be set to ``'drop'`` using:meth:`set_params`... versionchanged:: 0.21 ``'drop'`` is accepted. Using None was deprecated in 0.22 and support was removed in 0.24.","[('lr', ...), ('svr', ...), ...]"
,"weights weights: array-like of shape (n_regressors,), default=NoneSequence of weights (`float` or `int`) to weight the occurrences ofpredicted values before averaging. Uses uniform weights if `None`.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for ``fit``.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting will be printed as itis completed... versionadded:: 0.23",False
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm.If none is given, 'rbf' will be used. If a callable is given it isused to precompute the kernel matrix.For an intuitive visualization of different kernel typessee :ref:`sphx_glr_auto_examples_svm_plot_svm_regression.py`",'rbf'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3


In [99]:
# stacking model

stacking_reg = StackingRegressor(
    estimators=[
        ('svr',reg_svr),
        ('rf', reg_rf),
        ('gb', reg_gb)
    ], 
    final_estimator=Ridge() # meta learner
)
stacking_reg

,"estimators estimators: list of (str, estimator)Base estimators which will be stacked together. Each element of thelist is defined as a tuple of string (i.e. name) and an estimatorinstance. An estimator can be set to 'drop' using `set_params`.","[('svr', ...), ('rf', ...), ...]"
,"final_estimator final_estimator: estimator, default=NoneA regressor which will be used to combine the base estimators.The default regressor is a :class:`~sklearn.linear_model.RidgeCV`.",Ridge()
,"cv cv: int, cross-validation generator, iterable, or ""prefit"", default=NoneDetermines the cross-validation splitting strategy used in`cross_val_predict` to train `final_estimator`. Possible inputs forcv are:* None, to use the default 5-fold cross validation,* integer, to specify the number of folds in a (Stratified) KFold,* An object to be used as a cross-validation generator,* An iterable yielding train, test splits,* `""prefit""`, to assume the `estimators` are prefit. In this case, the estimators will not be refitted.For integer/None inputs, if the estimator is a classifier and y iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used.In all other cases, :class:`~sklearn.model_selection.KFold` is used.These splitters are instantiated with `shuffle=False` so the splitswill be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.If ""prefit"" is passed, it is assumed that all `estimators` havebeen fitted already. The `final_estimator_` is trained on the `estimators`predictions on the full training set and are **not** cross validatedpredictions. Please note that if the models have been trained on the samedata to train the stacking model, there is a very high risk of overfitting... versionadded:: 1.1 The 'prefit' option was added in 1.1.. note:: A larger number of split will provide no benefits if the number of training samples is large enough. Indeed, the training time will increase. ``cv`` is not used for model evaluation but for prediction.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for `fit` of all `estimators`.`None` means 1 unless in a `joblib.parallel_backend` context. -1 meansusing all processors. See :term:`Glossary ` for more details.",None
,"passthrough passthrough: bool, default=FalseWhen False, only the predictions of estimators will be used astraining data for `final_estimator`. When True, the`final_estimator` is trained on the predictions as well as theoriginal training data.",False
,"verbose verbose: int, default=0Verbosity level.",0
,"kernel kernel: {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable, default='rbf'Specifies the kernel type to be used in the algorithm.If none is given, 'rbf' will be used. If a callable is given it isused to precompute the kernel matrix.For an intuitive visualization of different kernel typessee :ref:`sphx_glr_auto_examples_svm_plot_svm_regression.py`",'rbf'
,"degree degree: int, default=3Degree of the polynomial kernel function ('poly').Must be non-negative. Ignored by all other kernels.",3
,"gamma gamma: {'scale', 'auto'} or float, default='scale'Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.- if ``gamma='scale'`` (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,- if 'auto', uses 1 / n_features- if float, must be non-negative... versionchanged:: 0.22 The default value of ``gamma`` changed from 'auto' to 'scale'.",'scale'
,"coef0 coef0: float, default=0.0Independent term in kernel function.It is only significant in 'poly' and 'sigmoid'.",0.0
,"tol tol: float, default=1e-3Tolerance for stopping criterion.",0.001


### 5. Model Training and Evaluation
- Create a dictionary for all models
    - base learner
    - ensemble model
    - Prediction and visualization

In [100]:
# Dictionray of all model

model_to_train = {
    'Linear Regression': reg_lr,
    'Support Vector': reg_svr,
    'Random Forest' : reg_rf,
    'Gradient Boosting': reg_gb,
    'Voting Ensemble' : voting_reg,
    'Stacking Ensemble': stacking_reg
}


In [101]:
# Training and Evaluation

result = []

for name, model in model_to_train.items():
    # Create full pipeline with preprocessor
    
    pipe = Pipeline(
        [
            ('preprocessor', preprocessor), 
            ('model',model)
        ]
    )
    # train or fit the model
    pipe.fit(X_train, y_train)
    
    # predict
    y_pred = pipe.predict(X_test)
    
    # Evaluation 
    
    # r2 score
    r2 = r2_score(y_test, y_pred)
    
    # rmse
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    
    # mean absolute error
    mae = mean_absolute_error(y_test, y_pred)
    
    
    # append everything on result list
    result.append({
        "Model":name,
        "R2 Score": r2,
        "RMSE": rmse,
        "MAE": mae
    }) 
    

# Result : convert into dataframe

result_df = pd.DataFrame(result).sort_values("R2 Score", ascending= False)   


print(result_df)

               Model  R2 Score          RMSE          MAE
3  Gradient Boosting  0.900689   4271.894268  2535.062045
5  Stacking Ensemble  0.897674   4336.238800  2500.243749
2      Random Forest  0.880864   4678.877462  2586.373908
4    Voting Ensemble  0.838371   5449.792661  3241.152470
0  Linear Regression  0.806929   5956.342894  4177.045561
1     Support Vector  0.377436  10695.792622  5180.221495


### 6. Best model

In [116]:
# Best model
best_model_name = result_df.iloc[0]['Model']

best_model_obj = model_to_train[best_model_name]

print("Best Model Name: ", best_model_name)


Best Model Name:  Gradient Boosting


In [117]:
# Best model Evaluation Metrix
best_model_name = result_df.iloc[0]
best_model_name

Model       Gradient Boosting
R2 Score             0.900689
RMSE              4271.894268
MAE               2535.062045
Name: 3, dtype: object

In [124]:
# fit the best model

gb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model_obj)
])

gb_pipe.fit(X_train, y_train)

gb_pipe

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [125]:
# Prediction and visualization

# predict the final model

y_fianl_pred = gb_pipe.predict(X_test)

y_fianl_pred[:2]

array([10571.65062029,  7016.72589715])

In [126]:
#plot Actual vs predicted

plt.figure( figsize = (8,6) )

sns.scatterplot(x=y_test, y=y_fianl_pred, alpha = 0.6, color='teal' )
plt.plot( [2,5] , [2,5], color = "red", linestyle = '--'  )

plt.xlabel("Actual HSC Result")
plt.ylabel("Predicted HSC result")

plt.grid(True)
plt.show()

### 7. Cross Validation
- Model : Gradient Boosting

In [127]:
# Cross Validation with 5 fold
cv_scores = cross_val_score(gb_pipe,X_train, y_train, scoring = 'neg_mean_squared_error')

xg_rmse = np.sqrt(-cv_scores)

print("RMSE (GB): \n", xg_rmse)

RMSE (GB): 
 [5411.91100723 4540.96470013 4464.57924225 4685.08542233 4869.76460699]


In [129]:
print(f"RMSE: {xg_rmse.mean():.5f}")

print(f"STD: {xg_rmse.std():.5f}")

RMSE: 4794.46100
STD: 338.21901


### 8. Grid Search CV on Our model
- To Get the best Parameter

In [150]:
#  grid for GradientBoostingRegressor
# (more focused + better results on average)

param_grid = {
    'model__n_estimators':     [100, 150, 200],      
    'model__learning_rate':    [0.01, 0.08, 0.1],   
    'model__max_depth':        [3, 4, 5],               
    'model__min_samples_split': [4, 6, 8],                            
    'model__subsample':        [0.8,  1.0]             
    }

In [152]:
grid_search = GridSearchCV(
    estimator=gb_pipe,
    param_grid=param_grid,
    cv=5,                               
    scoring='neg_root_mean_squared_error',
    n_jobs= 1,                          
    verbose=2
)                  

# Fit
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=0.8; total time=   0.2s
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=0.8; total time=   0.1s
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=0.8; total time=   0.1s
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=0.8; total time=   0.1s
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=0.8; total time=   0.1s
[CV] END model__learning_rate=0.01, model__max_depth=3, model__min_samples_split=4, model__n_estimators=100, model__subsample=1.0; total time=   0.2s
[CV] END model__learning_rate=0.01, m

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'model__learning_rate': [0.01, 0.08, ...], 'model__max_depth': [3, 4, ...], 'model__min_samples_split': [4, 6, ...], 'model__n_estimators': [100, 150, ...], ...}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_root_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more m

In [153]:
# predict our model for Grid Search

y_pred_grid = grid_search.predict(X_test)

# r2 score
r2_grid = r2_score(y_test, y_pred_grid)

print("R2 Score (Grid Search): ", r2_grid)

R2 Score (Grid Search):  0.9070139932146066


In [154]:
# The best score
print(f"The Best Score (Grid Search): {- grid_search.best_score_}")

# The best parameter
print(f"\nThe Best Parameters(Grid Search):\n {grid_search.best_params_}")

The Best Score (Grid Search): 4687.555876675125

The Best Parameters(Grid Search):
 {'model__learning_rate': 0.08, 'model__max_depth': 3, 'model__min_samples_split': 8, 'model__n_estimators': 100, 'model__subsample': 0.8}


### 9. Randomized Search CV
    - To Get the best Parameter

In [158]:
# import  
from scipy.stats import randint

# define the parameters of randomized cv

param_dist = {
    'model__n_estimators' : randint(100,250) ,
    'model__max_depth': [None,10,20],
    'model__min_samples_split' : randint(1,10)
}

In [160]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    estimator = gb_pipe,
    param_distributions = param_dist,
    n_iter=1,
    cv = 5 ,
    scoring = 'neg_root_mean_squared_error',
    n_jobs = 1,
    verbose = 2,
    random_state = 42
)

# fit the Random Forest model with random search cv paramaters

random_search.fit(X_train,y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END model__max_depth=20, model__min_samples_split=4, model__n_estimators=192; total time=   0.9s
[CV] END model__max_depth=20, model__min_samples_split=4, model__n_estimators=192; total time=   0.9s
[CV] END model__max_depth=20, model__min_samples_split=4, model__n_estimators=192; total time=   0.7s
[CV] END model__max_depth=20, model__min_samples_split=4, model__n_estimators=192; total time=   0.9s
[CV] END model__max_depth=20, model__min_samples_split=4, model__n_estimators=192; total time=   0.8s


,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Pipeline(step...m_state=42))])
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'model__max_depth': [None, 10, ...], 'model__min_samples_split': <scipy.stats....0016DC23A6010>, 'model__n_estimators': <scipy.stats....0016DC2654190>}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",1
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'neg_root_mean_squared_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-valid

In [161]:
# predict our model for Random Search

y_pred_random = random_search.predict(X_test)

# r2 score
r2_random = r2_score(y_test, y_pred_random)

print("R2 Score (Random Search): ", r2_random)

R2 Score (Random Search):  0.8226408830393308


In [162]:
# The best score
print(f"The Best Score (Random Search): {- random_search.best_score_}")

# The best parameter
print(f"\nThe Best Parameters(Random Search):\n {random_search.best_params_}")

The Best Score (Random Search): 5954.356700369516

The Best Parameters(Random Search):
 {'model__max_depth': 20, 'model__min_samples_split': 4, 'model__n_estimators': 192}


### 10. Comapre Which Parameters is Best for Our model.

- 10.1: Prediction and Evaluation for Grid search:

    - The Best Parameters(Grid Search):
        - 'model__learning_rate': 0.08, 
        - 'model__max_depth': 3, 
        - 'model__min_samples_split': 8, 
        - 'model__n_estimators': 100, 
        - 'model__subsample': 0.8

    - R2 Score (Grid Search):  0.9070139932146066
    - The Best Score (Grid Search): 4687.555876675125
    

- 10.2: Prediction and Evaluation for Random Search cv:

    - The Best Parameters(Random Search):
        - 'model__max_depth': 20, 
        - 'model__min_samples_split': 4, 
        - 'model__n_estimators': 192

    - R2 Score (Random Search):  0.8226408830393308
    - The Best Score (Random Search): 5954.356700369516


- So, the Final Parameters(Grid Serarch cv=5):
    - 'model__learning_rate': 0.08, 
    - 'model__max_depth': 3, 
    - 'model__min_samples_split': 8, 
    - 'model__n_estimators': 100, 
    - 'model__subsample': 0.8


### 11. Save Model with Best Parameters

In [163]:
# Save the Final MOdel
import pickle
filename = "gb_pipe.pkl"

with open( filename, "wb" ) as file:
  pickle.dump( gb_pipe, file )

In [164]:
# load the final model 

with open("gb_pipe.pkl" , "rb") as file:
    model_final_gb = pickle.load(file)
    
model_final_gb

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

### 12. MLFlow Introduce
- 12.1: Discover MLFlow

- 12.2: Mlflow with RandomForest model

- 12.3: Mlflow with our best model

#### 12.1: Discover MLFlow

In [167]:
import mlflow


# Tracking folder
mlflow.set_tracking_uri("mlruns_Charges")

# experiment name
mlflow.set_experiment("Medical Insurance Charges")

2026/01/18 22:39:42 INFO mlflow.tracking.fluent: Experiment with name 'Medical Insurance Charges' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///e:/Project/Health-Insurance-Cost-Predictor/mlruns_Charges/858796445197747756', creation_time=1768754382650, experiment_id='858796445197747756', last_update_time=1768754382650, lifecycle_stage='active', name='Medical Insurance Charges', tags={}>

#### 12.2: Mlflow with RandomForest model

- The Final Parameters(Grid Serarch cv=5):
    - 'model__learning_rate': 0.08, 
    - 'model__max_depth': 3, 
    - 'model__min_samples_split': 8, 
    - 'model__n_estimators': 100, 
    - 'model__subsample': 0.8


In [168]:
# the best parameter of Random Forest
best_params = {
    'n_estimators': 100,
    'learning_rate': 0.08,
    'max_depth': 3,
    'min_samples_split': 8,
    'subsample': 0.8,
    'random_state': 42
}

#### 12.3: Mlflow with our best model
- Save The Best model and Best Parameters 

In [169]:
# Start Ml flow
import mlflow.sklearn

with mlflow.start_run(run_name= "model_final_gb"):
    ### === log parameters === ###
    mlflow.log_params(best_params)
    
    ### === log Model name === ###
    mlflow.log_param("model_type", "GradientBoostingRegressor")
    
    ### === Train the model  === ###
    model_final_gb.fit(X_train,y_train)
   
    
    # Train predict (To check overfit or not)
    y_train_pred = model_final_gb.predict(X_train)
    
    # Train Evaluation
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    
    train_r2 = r2_score(y_train, y_train_pred)
    
    ### === log train details === ###
    mlflow.log_metric("Final model train rmse", train_rmse)
    mlflow.log_metric("Final model train r2", train_r2)
    
    
    
    ### === Test Predict === ###
    # predict
    y_test_pred = model_final_gb.predict(X_test)
    
    # Evaluation
    
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    test_r2 = r2_score(y_test, y_test_pred)
    
    ### === Log the test detatis === ###
    mlflow.log_metric("Final Model Test rmse", test_rmse)
    mlflow.log_metric("Final Model Test r2", test_r2)


In [170]:
print("Train R2 socre: ", train_r2)
print("Train RMSE socre: ", train_rmse)


print("\nTest R2 socre: ", test_r2)
print("Test RMSE socre: ", test_rmse)


print("\nBest Parameters: ")
best_params

Train R2 socre:  0.8932251074377553
Train RMSE socre:  3823.4865230896967

Test R2 socre:  0.9006885528992274
Test RMSE socre:  4271.894267589475

Best Parameters: 


{'n_estimators': 100,
 'learning_rate': 0.08,
 'max_depth': 3,
 'min_samples_split': 8,
 'subsample': 0.8,
 'random_state': 42}